In [1]:
import os
import random
import shutil
import tensorflow as tf
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras import *
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from shutil import copyfile
from os import getcwd
from os import listdir
from tensorflow.keras.layers import *
from tensorflow.keras.models import *
from tensorflow.keras.callbacks import *
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image  as mpimg

# Step 1: Chose a prebuilt model

There are currently 3 models:

 + A basic nureal net
 + InceptionV3
 + MobileNetV2
 
 If you don't know which one, the fastest and best is usually MobileNetV2

### Basic Nuralnet Model

In [2]:
model = tf.keras.models.load_model('mask_detector_basic.model')\
labels_dict={0:'without_mask',1:'with_mask'}
color_dict={0:(0,0,255),1:(0,255,0)}

### MobileNetV2 Model (Recommended)

In [3]:
model = tf.keras.models.load_model('mask_detector_mobilenet.model')
labels_dict={1:'without_mask',0:'with_mask'}
color_dict={1:(0,0,255),0:(0,255,0)}

### InceptionV3 Model

In [4]:
model = tf.keras.models.load_model("mask_detector_inceptionv3.model")
labels_dict={1:'without_mask',0:'with_mask'}
color_dict={1:(0,0,255),0:(0,255,0)}

# Step 2: Use the model on the webcam feed

In [23]:
import cv2
face_clsfr=cv2.CascadeClassifier('haarcascade_frontalface_default.xml')


size = 4
webcam = cv2.VideoCapture(0) #Use camera 0

# We load the xml file for the face detection
classifier = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

while True:
    (rval, im) = webcam.read()
    im=cv2.flip(im,1,1) #Flip to act as a mirror

    # Resize the image to speed up detection
    mini = cv2.resize(im, (im.shape[1] // size, im.shape[0] // size))

    # detect MultiScale / faces 
    faces = classifier.detectMultiScale(mini)

    # Draw rectangles around each face
    for f in faces:
        
        (x, y, w, h) = [v * size for v in f] #Scale the shapesize backup
        cv2.putText(im,str(faces), (50, 50),cv2.FONT_HERSHEY_SIMPLEX,0.5,(255,255,255),2)
 
        #Save just the rectangle faces in SubRecFaces
        face_img = im[y-30:y+h+30, x-30:x+w+30]
        resized=cv2.resize(face_img,(200,200))
        normalized=resized/255.0
        reshaped=np.reshape(normalized,(1,200,200,3))
        reshaped = np.vstack([reshaped])
        result=model.predict(reshaped)
        #print(result)
        
        label=np.argmax(result,axis=1)[0]
      
        cv2.rectangle(im,(x,y),(x+w,y+h),color_dict[label],2)
        cv2.rectangle(im,(x,y-40),(x+w,y),color_dict[label],-1)
        cv2.putText(im, labels_dict[label] + str(round(result[0][label],2)), (x, y-10),cv2.FONT_HERSHEY_SIMPLEX,0.8,(255,255,255),2)
        
    # Show the image
    cv2.imshow('Mask detector',  im)
    key = cv2.waitKey(10)
    # if Esc key is press then break out of the loop 
    if key == 27: #The Esc key
        break
# Stop video
webcam.release()

# Close all started windows
cv2.destroyAllWindows()

## Below is how the models were built. Feel free to modify as you see fit.

You'll first need to clone the dataset if you want to rebuild the model.

In [ ]:
!git clone https://github.com/prajnasb/observations.git

In [5]:
#Import the dataset as an ImageDataGenerator

TRAINING_DIR = "observations/experiements/data/"
train_datagen = ImageDataGenerator(rescale=1.0/255,
                                   rotation_range=40,
                                   width_shift_range=0.3,
                                   height_shift_range=0.3,
                                   shear_range=0.3,
                                   zoom_range=0.3,
                                   horizontal_flip=True,
                                   fill_mode='nearest')

train_generator = train_datagen.flow_from_directory(TRAINING_DIR, 
                                                    batch_size=10, 
                                                    target_size=(200,200))


Found 1376 images belonging to 2 classes.


## + Simple NeuralNet

In [6]:
#Define a basic neural net with binary return values.
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(100, (3,3), activation='relu', input_shape=(200, 200, 3)),
    tf.keras.layers.MaxPooling2D(2,2),
    
    tf.keras.layers.Conv2D(100, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    
    tf.keras.layers.Conv2D(100, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Dense(1024, activation='relu'),
    tf.keras.layers.Dense(2, activation='softmax')
])
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])

In [7]:
#Build the model...
history = model.fit_generator(train_generator,
                              epochs=10)

#and save it

model.save("mask_detector_basic.model", save_format="h5")

# then go try the camera feed like above!

/home/pinchas/anaconda3/envs/main/lib/python3.9/site-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/10
138/138 [==============================] - 17s 114ms/step - loss: 0.8233 - acc: 0.5368
Epoch 2/10
138/138 [==============================] - 15s 109ms/step - loss: 0.6904 - acc: 0.5230
Epoch 3/10
138/138 [==============================] - 15s 109ms/step - loss: 0.6906 - acc: 0.5519
Epoch 4/10
138/138 [==============================] - 15s 109ms/step - loss: 0.6423 - acc: 0.6632
Epoch 5/10
138/138 [==============================] - 15s 110ms/step - loss: 0.4971 - acc: 0.7729
Epoch 6/10
138/138 [==============================] - 15s 109ms/step - loss: 0.4426 - acc: 0.81511s - loss: 0.
Epoch 7/10
138/138 [==============================] - 15s 109ms/step - loss: 0.3362 - acc: 0.8664
Epoch 8/10
138/138 [==============================] - 15s 110ms/step - loss: 0.3563 - acc: 0.8551
Epoch 9/10
138/138 [==============================] - 15s 109ms/step - loss: 0.3723 - acc: 0.8385
Epoch 10/10
138/138 [==============================] - 15s 109ms/step - loss: 0.3340 - acc: 0.8740


## + MobileNetV2

In [13]:
#Import MobileNet using the imagenet weights
from tensorflow.keras.applications import MobileNetV2

baseModel = MobileNetV2(weights="imagenet", include_top=False,
    input_shape=(200, 200,3))

In [14]:
#Adding the end layers to the model
headModel = baseModel.output
headModel = AveragePooling2D(pool_size=(5, 5))(headModel)
headModel = Flatten(name="flatten")(headModel)
headModel = Dense(1024, activation="relu")(headModel)
headModel = Dropout(0.3)(headModel)
headModel = Dense(2, activation="softmax")(headModel)

#Freeze the imported layers
model = Model(inputs=baseModel.input, outputs=headModel)
for layer in baseModel.layers:
    layer.trainable = False

#Compile the model
model.compile(loss="binary_crossentropy", optimizer='adam')

In [16]:
#build 
history = model.fit_generator(train_generator,
                              epochs=10)
# and save
model.save("mask_detector_mobilenet.model", save_format="h5")

Epoch 1/10
138/138 [==============================] - 16s 101ms/step - loss: 0.3492
Epoch 2/10
138/138 [==============================] - 14s 100ms/step - loss: 0.1642
Epoch 3/10
138/138 [==============================] - 14s 99ms/step - loss: 0.1153
Epoch 4/10
138/138 [==============================] - 14s 100ms/step - loss: 0.0978
Epoch 5/10
138/138 [==============================] - 14s 99ms/step - loss: 0.0832
Epoch 6/10
138/138 [==============================] - 15s 109ms/step - loss: 0.0770
Epoch 7/10
138/138 [==============================] - 14s 104ms/step - loss: 0.0849
Epoch 8/10
138/138 [==============================] - 14s 104ms/step - loss: 0.0806
Epoch 9/10
138/138 [==============================] - 14s 103ms/step - loss: 0.0675
Epoch 10/10
138/138 [==============================] - 14s 105ms/step - loss: 0.0788


## + InceptionV3

In [20]:
#Import InceptionV3 using the imagenet weights
from tensorflow.keras.applications.inception_v3 import InceptionV3

pre_trained_model = InceptionV3(input_shape=(200, 200, 3),
 include_top=False,
 weights="imagenet")

#lock the premade layers
for layer in pre_trained_model.layers:
    layer.trainable = False
last_layer = pre_trained_model.get_layer('mixed7')
print('last layer output shape: ', last_layer.output_shape)
last_output = last_layer.output

#define the final layers
x = tf.keras.layers.Flatten()(last_output)
x = tf.keras.layers.Dropout(0.3)(x)
x = tf.keras.layers.Dense(512, activation='relu')(x)
x = tf.keras.layers.Dense(1, activation='sigmoid')(x)

#Compile the model
model.compile(loss="binary_crossentropy", optimizer='adam')

last layer output shape:  (None, 10, 10, 768)


In [21]:
history = model.fit_generator(train_generator,
                              epochs=10)


Epoch 1/10
138/138 [==============================] - 16s 99ms/step - loss: 0.0901
Epoch 2/10
138/138 [==============================] - 14s 99ms/step - loss: 0.2101
Epoch 3/10
138/138 [==============================] - 14s 99ms/step - loss: 0.0984
Epoch 4/10
138/138 [==============================] - 14s 99ms/step - loss: 0.1260
Epoch 5/10
138/138 [==============================] - 14s 100ms/step - loss: 0.0624
Epoch 6/10
138/138 [==============================] - 15s 112ms/step - loss: 0.1173
Epoch 7/10
138/138 [==============================] - 14s 101ms/step - loss: 0.0842
Epoch 8/10
138/138 [==============================] - 14s 99ms/step - loss: 0.1040
Epoch 9/10
138/138 [==============================] - 14s 99ms/step - loss: 0.0558
Epoch 10/10
138/138 [==============================] - 14s 99ms/step - loss: 0.0787


In [22]:
model.save("mask_detector_inceptionv3.model", save_format="h5")